In [2]:
%reload_ext autoreload
%autoreload 2
    
import sys
import random
import os
import pickle
repo_dir = '/srv/local/work/swang141/Sheng_repo/'
sys.path.append(repo_dir)
os.chdir(repo_dir)

from pikachu.datasets.WordNet import parse_sentence
import operator
import collections
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn.linear_model import LogisticRegression
import numpy as np

GO_term_dir = 'data/Pubmed/GO_sentences/'
if not os.path.exists(GO_term_dir):
    os.makedirs(GO_term_dir)
    
GO_term_file = 'data/NLP_Dictionary/go_term.txt'
GO_term = set()

fin = open(GO_term_file)
for line in fin:
    GO_term.add(line.lower().strip())
fin.close()

#GO_term = set()
#GO_term.add('mll1 complex')
#GO_term.add('binding')
nchunk = 100
pubmed_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/data/pubmed/'

GO_term_ct = {}
GO2text = {}

sent_collection = []
negative_set = []
negative_thres = 0.999
#print self.pubmed_dir
ct = 0
for pid in range(nchunk):
    fin = open(pubmed_dir+'pmid2meta_autophrase.chunk'+str(pid))
    for line in fin:
        text_l = line.lower().translate(None,',?!:()=%>/[]').strip().strip('.').split('. ')
        #text_l = [line.lower().translate(None,',?!:()=%>/[]').strip().strip('.')]
        '''
        for text in text_l:
            pset = parse_sentence(GO_term,text,max_phrase_length = 5)
            for p in pset:
                if p not in GO2text:
                    GO2text[p] = {}
                GO2text[p][text] = 1
                #print text
            r = random.uniform(0, 1)
            if r>negative_thres:
                negative_set.append(text) 
        '''
        ct += 1
    fin.close()
    print pid,len(negative_set),ct
negative_set = np.array(negative_set)



0 0 41828
1 0 83656
2 0 125484
3 0 167312
4 0 209140
5 0 250968
6 0 292796
7 0 334624
8 0 376452
9 0 418280
10 0 460108
11 0 501936
12 0 543764
13 0 585592
14 0 627420
15 0 669248
16 0 711076
17 0 752904
18 0 794732
19 0 836560
20 0 878388
21 0 920216
22 0 962044
23 0 1003872
24 0 1045700
25 0 1087528
26 0 1129356
27 0 1171184
28 0 1213012
29 0 1254840
30 0 1296668
31 0 1338496
32 0 1380324
33 0 1422152
34 0 1463980
35 0 1505808
36 0 1547636
37 0 1589464
38 0 1631292
39 0 1673120
40 0 1714948
41 0 1756776
42 0 1798604
43 0 1840432
44 0 1882260
45 0 1924088
46 0 1965916
47 0 2007744
48 0 2049572
49 0 2091400
50 0 2133228
51 0 2175056
52 0 2216884
53 0 2258712
54 0 2300540
55 0 2342368
56 0 2384196
57 0 2426024
58 0 2467852
59 0 2509680
60 0 2551508
61 0 2593336
62 0 2635164
63 0 2676992
64 0 2718820
65 0 2760648
66 0 2802476
67 0 2844304
68 0 2886132
69 0 2927960
70 0 2969788
71 0 3011616
72 0 3053444
73 0 3095272
74 0 3137100
75 0 3178928
76 0 3220756
77 0 3262584
78 0 3304412
79 0 334

In [123]:
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn import linear_model
from pikachu.datasets.WordNet import parse_sentence
import operator
import collections
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn.linear_model import LogisticRegression
import numpy as np

GO2keyword = {}
nkeyword = 20

e = list(GO2text['dna repair'])

GO_predict_dir = 'data/Pubmed/predict_GO/'
if not os.path.exists(GO_predict_dir):
    os.makedirs(GO_predict_dir)
nmax_train_data = 3000
nmin_train_data = 3000
for C in [10]:
    for ct,GO in enumerate(GO_term):
        if GO not in GO2text:
            continue
        if GO!= 'small molecular binding':
            continue
        GO_term_file = GO_term_dir + GO.replace('/','_').replace(' ','_') +'.pkl'
        if len(GO2text[GO]) > nmax_train_data:
            select_sent = np.random.choice(GO2text[GO].keys(), nmax_train_data, replace=False)
            GO2text[GO] = {}
            for t in select_sent:
                GO2text[GO][t] = 1
        nneg_data = min(max(nmin_train_data,len(GO2text[GO])*9), len(negative_set))

        text_data = []
        for t in GO2text[GO]:
            text_data.append((t.replace(GO,''),1))    
        text_data_neg = np.random.choice(negative_set, nneg_data, replace=False)

        for text in text_data_neg:
            if text not in GO2text[GO]:
                text_data.append((text.replace('dna repair',''),0))
        text_data = np.array(text_data)
        np.random.shuffle(text_data)
        train_data = text_data[:np.int(0.99*len(text_data))]
        test_data = text_data[np.int(0.99*len(text_data)):]
        has_pos = False
        for t in train_data:
            if t[1] == '1':
                has_pos = True
                break
        if not has_pos:
            continue
        #lr = LogisticRegression(penalty='l1',C=1) #
        lr = LogisticRegression()
        text_clf = TextClassify()
        text_clf.train(lr,train_data)
        
        
        test_acc = text_clf.evaluate(test_data)
        train_acc = text_clf.evaluate(train_data)
        print GO,len(GO2text[GO]),test_acc,train_acc,text_clf.nword,len(np.nonzero(text_clf.clf.coef_)[1]),C
        para = text_clf.clf.coef_[0]*-1
        word_ind = np.argsort(para)[:20]
        for i in range(20):
            wi = word_ind[i]
            print i,text_clf.i2w[wi],text_clf.feat_wt[wi],text_clf.clf.coef_[0][wi]
        print '-------split line-------'
        '''
        with open(GO_term_file, 'wb') as output:
            pickle.dump(text_clf, output, pickle.HIGHEST_PROTOCOL)
        if ct%100==0:
            print 'finished',ct*1.0/len(GO_term),ct,text_clf.nword

        text_file = GO_predict_dir + GO.replace('/','_').replace(' ','_')+'.txt'
        fout = open(text_file,'w')
        text_clf = pickle.load(open(GO_term_file, "rb" ))
        GO2keyword[GO] = text_clf.get_keyword(nkeyword)
        print GO, len(GO2text[GO]),GO2keyword[GO]

        nsent = 0
        #print self.pubmed_dir
        for pid in range(nchunk):
            fin = open(pubmed_dir+'pmid2meta_autophrase.chunk'+str(pid))
            for line in fin:
                text_l = line.lower().translate(None,',?!:()=%>/[]').strip().strip('.').split('. ')
                for text in text_l:
                    pset = parse_sentence(GO2keyword[GO],text,max_phrase_length = 5)
                    if len(pset) > 0 and GO not in text:
                        lab = text_clf.predict(text)
                        if lab==1:
                            prob = text_clf.predict_prob(text)
                            print prob,text
                            fout.write(str(prob)+'\t'+text+'\n')
                            nsent += 1
            fin.close()
            print pid, nsent, len(GO2text[GO])
        fout.close()
        print GO, nsent
        '''


In [116]:
np.shape(train_data)

(7007, 2)

In [101]:
np.where(feat!=0)

(array([   13,    20,    33,    34,    49,    63,    85,    89,    98,
          114,   143,   253,   419,   445,   460,   481,  1511,  2123,
         2340,  3523,  5532,  5583,  9042, 10744]),)

In [5]:
4182800*4/1000/1000.

16.731

In [2]:
import multiprocessing
import sys
import random
import os
import pickle
import operator
import collections
import numpy as np
import time


def write_GO_to_file(pid, GO_sent_dir, GO2text):
    cal_feat_time = 0.
    pickle_time = 0.
    cal_prob_time = 0.
    max_abs = 50
    for ct,GO in enumerate(GO2text):
        sent2prob = {}
        sent_file = GO_sent_dir + str(pid)+'/'+ GO.replace('/','_').replace(' ','_')+'.txt'

        pred_file =    GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
        start = time.time()
        text_clf = pickle.load(open(pred_file, "rb" ))
        end = time.time()
        pickle_time += end - start
        text2prob = {}
        for text in GO2text[GO]:
            '''
            start = time.time()
            feat = [text_clf.get_test_feature(text)]
            end = time.time()
            cal_feat_time += end - start
            start = time.time()
            #prob = text_clf.predict_prob(text)
            #prob = text_clf.predict_prob_feat(feat)
            feat_pos = np.nonzero(text_clf.clf.coef_)[1]
            sc = text_clf.clf.intercept_[0]
            for f in feat_pos:
                sc += text_clf.clf.coef_[0][f] * feat[0][f]
            prob = 1 / (1 + np.exp(-1*sc))
            prob1 = text_clf.predict_prob_feat(feat)
            print prob,prob1
            end = time.time()
            cal_prob_time += end - start
            '''
            prob = text_clf.predict_prob_fast(text)
            if prob > 0.99:
                text2prob[text] = prob*-1
                #fout.write(text+'\t'+str(prob)+'\n')
        fout = open(sent_file,'w')
        text2prob_sort = sorted(text2prob.items(), key = lambda item:item[1])
        noutput = min(max_abs, len(text2prob_sort))
        for i in range(noutput):
            fout.write(text2prob_sort[i][0]+'\t' + str(text2prob_sort[i][1])+'\n')
        fout.close()
        if ct%100==0:
            print 'sent2prob',ct*1.0/len(GO2text)
            sys.stdout.flush()


def predict_GO_term(proc_id, keyword2GO,p2chunk,GO_sent_dir,GO_pred_dir):
    keyword_set = set(keyword2GO.keys())
    print len(keyword_set)
    parse_time = 0.
    append_time = 0.
    nsent = 0
    GO2text = {}
    #print self.pubmed_dir
    for pid in p2chunk:
        fin = open(pubmed_dir+'pmid2meta_autophrase.chunk'+str(pid))
        for ct,line in enumerate(fin):
            text_l = [line.lower().translate(None,',?!:()=%>/[]').strip().strip('.')]
            for text in text_l:
                start = time.time()
                pset = parse_sentence(keyword_set,text,max_phrase_length = 1)
                end = time.time()
                parse_time += end - start
                start = time.time()
                cur_GO = set()
                for p in pset:
                    for GO in keyword2GO[p]:
                        if GO in text:
                            continue
                        if GO in cur_GO:
                            continue
                        cur_GO.add(GO)
                        if GO not in GO2text:
                            GO2text[GO] = []
                        GO2text[GO].append(text)
                        #fout.write(str(prob)+'\t'+text+'\n')
                        nsent += 1
                end = time.time()
                append_time += end - start
            if ct%1000==0:
                print proc_id, pid, 'read finished',ct/40000.
                sys.stdout.flush()
        print proc_id, pid,'start to write file',parse_time,append_time
        sys.stdout.flush()
        write_GO_to_file(proc_id,GO_sent_dir,GO2text)
        GO2text = {}
        print proc_id, pid,'end to write file'
        sys.stdout.flush()
        fin.close()

def generate_train_model(list, n):
    N = len(list)
    ind = np.random.choice(N, n)
    new_list = []
    for i in ind:
        new_list.append(list[i])
    return new_list
    #for i in range(n):
    #    yield np.random.choice(list, 1)

def train_model(GO, GO2text, text_file, pred_file, log_file, nfold = 3):
    print GO,'train'
    sys.stdout.flush()
    GO2keyword = {}
    nmax_train_data = 3000
    nmin_train_data = 3000

    train_data = []
    if len(GO2text) > nmax_train_data:
        select_sent = generate_train_model(GO2text.keys(), nmax_train_data)
        for t in select_sent:
            train_data.append((t[0].replace(GO,''),1))
    else:
        for t in GO2text:
            train_data.append((t.replace(GO,''),1))
    sys.stdout.flush()
    ntrain_data = min(nmin_train_data, len(negative_set))
    train_data_neg = generate_train_model(negative_set, ntrain_data)
    print GO,len(train_data),'positive samples'
    sys.stdout.flush()
    for text in train_data_neg:
        if text[0] not in GO2text:
            train_data.append((text[0],0))
    print GO,len(train_data),'all samples'
    sys.stdout.flush()
    lr = LogisticRegression()#penalty='l1',C=0.1
    text_clf = TextClassify()
    
    train_data = np.array(train_data)
    cv_data = train_data
    np.random.shuffle(cv_data)
    ratio = 1 - 1.0 / nfold
    cv_train_data = cv_data[:np.int(ratio*len(cv_data))]
    cv_test_data = cv_data[np.int(ratio*len(cv_data)):]
    fout = open(log_file,'w')
    has_pos = False
    for t in cv_train_data:
        if t[1] == '1':
            has_pos = True
            break    
    if has_pos:
        text_clf.train(lr,cv_train_data)
        test_acc = text_clf.evaluate(cv_test_data)
        train_acc = text_clf.evaluate(cv_train_data)        
        fout.write(str(test_acc)+'\t'+str(train_acc)+'\t'+str(text_clf.nword)+'\n')
        para = text_clf.clf.coef_[0]
        word_ind = np.argsort(para)[:20]
        for i in range(20):
            wi = word_ind[i]
            fout.write(str(i)+'\t'+str(text_clf.i2w[wi])+'\t'+str(text_clf.clf.coef_[0][wi])+'\n')
    #lr = LogisticRegression(penalty='l1',C=1) #    
    #print test_acc, train_acc,text_clf.nword    
    text_clf.train(lr,train_data)
    para = text_clf.clf.coef_[0]*-1
    word_ind = np.argsort(para)[:20]
    for i in range(20):
        wi = word_ind[i]
        fout.write(str(i)+'\t'+str(text_clf.i2w[wi])+'\t'+str(text_clf.clf.coef_[0][wi])+'\n')
    sys.stdout.flush()
    fout.close()
    with open(pred_file, 'wb') as output:
        pickle.dump(text_clf, output, pickle.HIGHEST_PROTOCOL)


def call_merge_preprocess(GO_term_set,GO_text_dir,GO_pred_dir,GO_sent_dir,negative_set):
    for ct, GO in enumerate(GO_term_set):
        text_file = GO_text_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
        if not os.path.isfile(text_file):
            continue
        print GO
        sys.stdout.flush()
        GO2text = pickle.load(open(text_file, "rb" ))
        pred_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
        log_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.log'
        if not os.path.isfile(pred_file):
            print GO,len(GO2text)
            sys.stdout.flush()
            #try:
            train_model(GO,GO2text,text_file,pred_file,log_file)
            #except Exception, e:
            #    continue
        if not os.path.isfile(pred_file):
            continue
        text_clf = pickle.load(open(pred_file, "rb" ))

def call_predict_preprocess(i,npid,p2chunk,GO_term_set,GO_pred_dir,GO_sent_dir):
    start = time.time()
    keyword2GO = {}
    nkeyword = 20
    ct = 0
    print len(GO_term_set)
    sys.stdout.flush()
    for ct, GO in enumerate(GO_term_set):
        pred_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
        if not os.path.isfile(pred_file):
            continue
        text_clf = pickle.load(open(pred_file, "rb" ))
        key_word = text_clf.get_keyword(nkeyword)
        print GO,key_word
        sys.stdout.flush()
        for w in key_word:
            if w not in keyword2GO:
                keyword2GO[w] = []
            keyword2GO[w].append(GO)
            ct += 1
        if ct%100==0:
            end = time.time()
            print 'len keyword',ct,end-start
            sys.stdout.flush()
    end = time.time()
    print 'len keyword',ct,end-start
    sys.stdout.flush()
    predict_GO_term(i, keyword2GO,p2chunk,GO_sent_dir,GO_pred_dir)

proc_id = 1
totalpid = 64
nchunk = 400
mode = 1#0: extract abs, 1: train_model, 2:generate sent
server = 'timan107'#'grenache'
if server == 'timan107':
    pubmed_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/data/pubmed/'
    GO_data_dir = '/srv/local/work/swang141/Sheng_repo/data/Pubmed/'
    GO_term_file = '/srv/local/work/swang141/Sheng_repo/data/NLP_Dictionary/go_term.txt'
    repo_dir = '/srv/local/work/swang141/Sheng_repo/'
else:
    pubmed_dir = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/pubmed/'
    GO_data_dir = '/cellar/users/majianzhu/Data/wangsheng/Sheng_repo/data/Pubmed/'
    GO_term_file = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/Sheng_repo/data/NLP_Dictionary/go_term.txt'
    repo_dir = '/data/cellardata/users/majianzhu/wangsheng/Sheng_repo/Sheng_repo/'



sys.path.append(repo_dir)
os.chdir(repo_dir)

from pikachu.datasets.WordNet import parse_sentence
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn.linear_model import LogisticRegression


GO_text_dir = GO_data_dir+'GO_abstracts_test/'
GO_pred_dir = GO_data_dir + 'GO_models_L2/'
GO_sent_dir = GO_data_dir + 'GO_sentences_test/'
GO_sent_merge_dir = GO_data_dir + 'GO_sentences_all/'

if not os.path.exists(GO_sent_merge_dir):
    os.makedirs(GO_sent_merge_dir)
if not os.path.exists(GO_sent_dir):
    os.makedirs(GO_sent_dir)
if not os.path.exists(GO_pred_dir):
    os.makedirs(GO_pred_dir)
if not os.path.exists(GO_text_dir):
    os.makedirs(GO_text_dir)


for i in range(totalpid):
    GO_sent_dir_pid = GO_sent_dir +str(i)+'/'
    if not os.path.exists(GO_sent_dir_pid):
        os.makedirs(GO_sent_dir_pid)


pred_GO_term = set()
abs_GO_term = set()
sent_GO_term = set()
fin = open(GO_term_file)
ct = 0
for line in fin:
    GO = line.lower().strip()
    text_file = GO_text_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
    pred_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
    sent_file = GO_sent_dir + GO.replace('/','_').replace(' ','_')+'.txt'
    if mode == 1 and not os.path.isfile(pred_file) and os.path.isfile(text_file):
        ct+=1
        if ct%totalpid == proc_id:
            pred_GO_term.add(GO)
    if mode == 0 and not os.path.isfile(text_file):
        ct+=1
        if ct%totalpid == proc_id:
            abs_GO_term.add(GO)
    if mode == 2 and not os.path.isfile(sent_file) and os.path.isfile(pred_file):
        ct+=1
        if ct%totalpid == proc_id:
            sent_GO_term.add(GO)
fin.close()
GO_term_ct = {}
GO2text = {}
print sent_GO_term

if mode==1 or mode == 0:
    sent_collection = []
    negative_set = []
    negative_thres = 0.99
    if len(abs_GO_term) > 0:
        for pid in range(nchunk):
            fin = open(pubmed_dir+'pmid2meta_autophrase.chunk'+str(pid))
            for line in fin:
                text_l = [line.lower().translate(None,',?!:()=%>/[]').strip().strip('.')]
                for text in text_l:
                    pset = parse_sentence(abs_GO_term,text,max_phrase_length = 5)
                    for p in pset:
                        if p not in GO2text:
                            GO2text[p] = {}
                        GO2text[p][text] = 1
            fin.close()
            print pid,len(negative_set)
            sys.stdout.flush()
        

    for ct, GO in enumerate(GO2text):
        GO2text_file = GO_text_dir+GO.replace('/','_').replace(' ','_')+'.pkl'
        if os.path.isfile(GO2text_file):
            continue
        with open(GO2text_file, 'wb') as output:
            pickle.dump(GO2text[GO], output, pickle.HIGHEST_PROTOCOL)

    for ct, GO in enumerate(pred_GO_term):
        text_file = GO_text_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
        if not os.path.isfile(text_file):
            continue
        GO2text = pickle.load(open(text_file, "rb" ))
        for text in GO2text:
            r = random.uniform(0, 1)
            if r>negative_thres:
                negative_set.append(text.replace(GO,''))
        print ct, len(pred_GO_term), len(negative_set)
        
    negative_set = np.array(negative_set)
    print 'read data finished number of negatives',len(negative_set)
    sys.stdout.flush()
    call_merge_preprocess(pred_GO_term,GO_text_dir,GO_pred_dir,GO_sent_dir,negative_set)
elif mode==2:
    pred_GO_term = set()
    abs_GO_term = set()
    sent_GO_term = set()
    fin = open(GO_term_file)
    ct = 0
    for line in fin:
        GO = line.lower().strip()
        sent_file = GO_sent_dir + str(proc_id)+'/'+ GO.replace('/','_').replace(' ','_')+'.txt'
        #if os.path.isfile(sent_file):
        #    continue
        sent_GO_term.add(GO)
        ct += 1
    print 'ngo',len(sent_GO_term)
    sys.stdout.flush()
    nchunk = 400
    p2chunk = {}
    for i in range(nchunk):
        pid = i%totalpid
        if pid not in p2chunk:
            p2chunk[pid] = set()
        p2chunk[pid].add(i)
    call_predict_preprocess(proc_id,totalpid,p2chunk[proc_id-1],sent_GO_term,GO_pred_dir,GO_sent_dir)
elif mode==3:
    sent_GO_term = set()
    fin = open(GO_term_file)
    ct = 0
    for line in fin:
        GO = line.lower().strip()
        sent_GO_term.add(GO)
        ct += 1
    for ct,GO in enumerate(sent_GO_term):
        print GO,ct
        new_sent_file = GO_sent_merge_dir +  GO.replace('/','_').replace(' ','_')+'.txt'
        fout = open(new_sent_file,'w')
        for i in range(totalpid):
            sent_file = GO_sent_dir + str(i)+'/'+ GO.replace('/','_').replace(' ','_')+'.txt'
            if not os.path.isfile(sent_file):
                continue
            fin = open(sent_file)
            for line in fin:
                fout.write(line.strip()+'\n')
            fin.close()
        fout.close()
else:
    pass


set([])
0 124 1
1 124 1
2 124 1
3 124 1
4 124 12
5 124 12
6 124 12
7 124 12
8 124 12
9 124 12
10 124 55
11 124 59
12 124 59
13 124 59
14 124 59
15 124 99
16 124 99
17 124 99
18 124 100
19 124 101
20 124 113
21 124 113
22 124 113
23 124 113
24 124 146
25 124 146
26 124 148
27 124 148
28 124 149
29 124 159
30 124 160
31 124 167
32 124 167
33 124 167
34 124 167
35 124 167
36 124 167
37 124 167
38 124 429
39 124 429
40 124 429
41 124 429
42 124 3326
43 124 3331
44 124 3332
45 124 3332
46 124 3355
47 124 3355
48 124 3358
49 124 3358
50 124 3358
51 124 3358
52 124 3358
53 124 3358
54 124 3358
55 124 3358
56 124 3358
57 124 3370
58 124 3370
59 124 3370
60 124 3372
61 124 3384
62 124 3385
63 124 3385
64 124 3385
65 124 3385
66 124 3385
67 124 3385
68 124 3385
69 124 3385
70 124 3385
71 124 3385
72 124 3411
73 124 3411
74 124 3412
75 124 3461
76 124 3461
77 124 3464
78 124 3476
79 124 3476
80 124 3476
81 124 3478
82 124 3478
83 124 3478
84 124 3478
85 124 3478
86 124 3478
87 124 3478
88 124 347

pikachu/models/text_classification/TextClassify.py:188: RuntimeWarning: divide by zero encountered in double_scalars
  self.feat_wt[i] = (np.sum(trainX[pos,i]) * 1.0 / len(pos) ) /(np.sum(trainX[neg,i]) * 1.0 / len(neg) )


platelet morphogenesis
platelet morphogenesis 3
platelet morphogenesis train
platelet morphogenesis 3 positive samples
platelet morphogenesis 3003 all samples
chromatoid body
chromatoid body 122
chromatoid body train
chromatoid body 122 positive samples
chromatoid body 3122 all samples
6-phospho-beta-galactosidase activity
6-phospho-beta-galactosidase activity 1
6-phospho-beta-galactosidase activity train
6-phospho-beta-galactosidase activity 1 positive samples
6-phospho-beta-galactosidase activity 3001 all samples


/srv/local/work/swang141/software/Anaconda/anaconda/lib/python2.7/site-packages/sklearn/metrics/ranking.py:547: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


endonuclease activity
endonuclease activity 1344
endonuclease activity train
endonuclease activity 1344 positive samples
endonuclease activity 4344 all samples
oxidative stress-induced premature senescence
oxidative stress-induced premature senescence 15
oxidative stress-induced premature senescence train
oxidative stress-induced premature senescence 15 positive samples
oxidative stress-induced premature senescence 3015 all samples
histone h3-k79 methylation
histone h3-k79 methylation 1
histone h3-k79 methylation train
histone h3-k79 methylation 1 positive samples
histone h3-k79 methylation 3001 all samples
immature t cell proliferation
immature t cell proliferation 1
immature t cell proliferation train
immature t cell proliferation 1 positive samples
immature t cell proliferation 3001 all samples
carboxy-lyase activity
carboxy-lyase activity 5
carboxy-lyase activity train
carboxy-lyase activity 5 positive samples
carboxy-lyase activity 3005 all samples


KeyboardInterrupt: 

In [44]:
%reload_ext autoreload
%autoreload 2
    
import sys
import random
import os
import pickle
repo_dir = '/srv/local/work/swang141/Sheng_repo/'
sys.path.append(repo_dir)
os.chdir(repo_dir)

from pikachu.datasets.WordNet import parse_sentence
import operator
import collections
from pikachu.models.text_classification.TextClassify import TextClassify
from sklearn.linear_model import LogisticRegression
import numpy as np

GO_term_dir = 'data/Pubmed/GO_sentences/'
if not os.path.exists(GO_term_dir):
    os.makedirs(GO_term_dir)
    
GO_term_file = 'data/NLP_Dictionary/go_term.txt'
GO_term = set()

fin = open(GO_term_file)
for line in fin:
    GO_term.add(line.lower().strip())
fin.close()

#GO_term = set()
#GO_term.add('mll1 complex')
#GO_term.add('binding')
nchunk = 100
pubmed_dir = '/srv/local/work/swang141/PatientSetAnnotation/Sheng/data/pubmed/'

GO_term_ct = {}
GO2text = {}

sent_collection = []
negative_set = []
negative_thres = 0.999
#print self.pubmed_dir
for pid in range(400):
    if pid !=311:
        continue
    fin = open(pubmed_dir+'pmid2meta_autophrase.chunk'+str(pid))
    for line in fin:
        text_l = line.lower().translate(None,',?!:()=%>/[]').strip().strip('.').split('. ')
        #text_l = [line.lower().translate(None,',?!:()=%>/[]').strip().strip('.')]
        for text in text_l:
            if 'new vaccine to be tested in britain nairobi' in text:
                print pid
                print text
                print line
                print len(text_l)
                break
    print pid


311
new vaccine to be tested in britain nairobi.
New vaccine to be tested in Britain, Nairobi.. UK vaccine test cautiously welcomed. HIV vaccine.. The impact of disease: two worlds meet.. Malaria: failure, puzzle, challenge.. Research for the people.. Health and medical practices in the pre-Columbian Americas.. Romania to stop medication for 10,000 HIV patients.. Thousands in the Americas infected by tainted blood.. Break the silence: Let's talk about AIDS.. UNAIDS director visits Denmark to discuss collaboration on AIDS crisis in Africa.. Global prevention, funding, accountability debated in fight against HIV / AIDS.. Pornographic lives.. Sexual abuse during childhood and adolescence as predictors of HIV-related sexual. Sexual violence in women's lives. Findings from the Women's Safety Project, a. The making of a whore.. Beijing women's conference will address AIDS.. SWAA mobilizes African women against AIDS.. Vaccination efforts lauded in Brazil.. Women in extreme poverty.. Organisin

311


In [32]:



nfold = 3

call_merge_preprocess(['oxidative stress-induced premature senescence'],GO_text_dir,GO_pred_dir,GO_sent_dir,negative_set)

for ct, GO in enumerate(['oxidative stress-induced premature senescence']):
    text_file = GO_text_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
    if not os.path.isfile(text_file):
        continue
    print GO
    sys.stdout.flush()
    GO2text = pickle.load(open(text_file, "rb" ))
    pred_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.pkl'
    log_file = GO_pred_dir + GO.replace('/','_').replace(' ','_')+'.log'

    print GO,'train'
    sys.stdout.flush()
    GO2keyword = {}
    nmax_train_data = 3000
    nmin_train_data = 3000

    train_data = []
    if len(GO2text) > nmax_train_data:
        select_sent = generate_train_model(GO2text.keys(), nmax_train_data)
        for t in select_sent:
            train_data.append((t[0].replace(GO,''),1))
    else:
        for t in GO2text:
            train_data.append((t.replace(GO,''),1))
    sys.stdout.flush()
    ntrain_data = min(nmin_train_data, len(negative_set))
    train_data_neg = generate_train_model(negative_set, ntrain_data)
    print GO,len(train_data),'positive samples'
    sys.stdout.flush()
    for text in train_data_neg:
        if text not in GO2text:
            train_data.append((text,0))
    print GO,len(train_data),'all samples'
    sys.stdout.flush()
    lr = LogisticRegression()#penalty='l1',C=0.1
    text_clf = TextClassify()

    train_data = np.array(train_data)
    cv_data = train_data
    np.random.shuffle(cv_data)
    ratio = 1 - 1.0 / nfold
    cv_train_data = cv_data[:np.int(ratio*len(cv_data))]
    cv_test_data = cv_data[np.int(ratio*len(cv_data)):]
    fout = open(log_file,'w')
    has_pos = False
    for t in cv_train_data:
        if t[1] == '1':
            has_pos = True
            break    
    if has_pos:
        text_clf.train(lr,cv_train_data)
        test_acc = text_clf.evaluate(cv_test_data)
        train_acc = text_clf.evaluate(cv_train_data)        
        print test_acc,train_acc,text_clf.nword
        para = text_clf.clf.coef_[0]*-1
        word_ind = np.argsort(para)[:20]
        for i in range(20):
            wi = word_ind[i]
            print i,text_clf.i2w[wi],text_clf.clf.coef_[0][wi],text_clf.feat_wt[wi]
    #lr = LogisticRegression(penalty='l1',C=1) #    
    #print test_acc, train_acc,text_clf.nword    
    text_clf.train(lr,train_data)
    para = text_clf.clf.coef_[0]*-1
    word_ind = np.argsort(para)[:20]
    for i in range(20):
        wi = word_ind[i]
        fout.write(str(i)+'\t'+str(text_clf.i2w[wi])+'\t'+str(text_clf.clf.coef_[0][wi])+'\n')
    sys.stdout.flush()
    fout.close()
    with open(pred_file, 'wb') as output:
        pickle.dump(text_clf, output, pickle.HIGHEST_PROTOCOL)
        

oxidative stress-induced premature senescence
oxidative stress-induced premature senescence
oxidative stress-induced premature senescence train
oxidative stress-induced premature senescence 15 positive samples
oxidative stress-induced premature senescence 3015 all samples


/srv/local/work/swang141/software/Anaconda/anaconda/lib/python2.7/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


0.9770459081836327 1.0 41942
0 senescence 0.8189817934527525 853.3125
1 oxidative 0.5760787650648997 36.75974025974026
2 premature 0.37796115638520666 83.25
3 mscs 0.33218515045861885 121.09090909090908
4 capabilities 0.3116390221510903 194.25
5 paeonol 0.30990672951474096 inf
6 occludin 0.30047590165496474 999.0
7 cells 0.28482830228646233 3.676656151419558
8 stress 0.2715233008457429 24.306569343065696
9 ptrfcavin-1 0.2650196367213048 inf
10 significantly 0.24907084780321292 3.488571428571428
11 zfp637 0.24819213742098933 inf
12 senescent 0.24286965032436192 360.75
13 h2o2 0.24187723481563958 309.2142857142857
14 protein 0.22539726401916213 2.8608247422680413
15 ovx 0.22262103547127904 208.125
16 against 0.2208758914274337 6.1213235294117645
17 expression 0.20261644115923919 2.3147964250248263
18 proliferative 0.18705343399409877 23.125
19 gcs 0.17813851102879888 inf


In [36]:
GO2text

{'autophagy through 4ebp1 and ampk regulates oxidative stress-induced premature. the aim of this study was to determine whether autophagy and ampk contribute to premature senescence in auditory cells. incubating hei-oc1 auditory cells with 5 mm h2o2 for 1 h induced senescence as demonstrated by senescence-associated beta-galactosidase sa-beta-gal staining. h2o2 treatment significantly delayed population-doubling time leaving cell viability unchanged. furthermore the proportion of sa-beta-gal-positive cells significantly increased. autophagy-related protein expression increased with atg7 and lc3-ii peaking 6 h and lamp2 peaking 24 h after h2o2 treatment. the expression of these proteins decreased 48 h after treatment. transmission electron microscopy revealed lipofuscin and aggregates within autolysosomes which accumulated markedly in the cytoplasm of hei-oc1 cells 48 h after treatment. akt and p70s6 phosphorylation markedly decreased after h2o2 treatment but 4ebp1 phosphorylation signi

In [28]:
 np.exp(s)

0.3193135828470574

In [30]:
lr.intercept_[0]

-0.12873856858812432